In [290]:
import transformers
from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer, pipeline, BertForTokenClassification, BertTokenizer
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from gensim.utils import simple_preprocess
import re

In [330]:
def sent_to_words(sent):
    sent = re.sub("\s+", " ", sent)  # remove newline chars
    sent = re.sub("\'", "", sent)  # remove single quotes
    sent = re.sub("  ", " ", sent)  # remove single quotes
    #sent = simple_preprocess(str(sent), deacc=True) 
    return sent

In [2]:
MODELS_DIR = [
    r"pucpr/clinicalnerpt-diagnostic",
    r"pucpr/clinicalnerpt-disease",           
    r"pucpr/clinicalnerpt-sign",
    r"pucpr/clinicalnerpt-disorder",             
    r"pucpr/clinicalnerpt-finding"   
]

In [272]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForTokenClassification, BertConfig
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize  
from transformers import pipeline


def predictBERTNER(sentencas, MODEL_DIR):
        
    model = BertForTokenClassification.from_pretrained(MODEL_DIR)
    #tokenizer = BertTokenizer.from_pretrained(MODEL_DIR, do_lower_case=True) # lower or not, this is important
    tokenizer = BertTokenizer.from_pretrained("pucpr/biobertpt-all", do_lower_case=True, truncation=True) # lower or not, this is important
    #config = BertConfig.from_pretrained(MODEL_DIR, num_labels=3)
    config = BertConfig.from_pretrained(MODEL_DIR)
    #print(config.id2label)
    #print(type(config.id2label))
    dic_label=config.id2label
    
    predictedModel=[]
    
    for test_sentence in sentencas:
        tokenized_sentence = tokenizer.encode(test_sentence)
        input_ids = torch.tensor([tokenized_sentence])#.cuda()
        
        with torch.no_grad():
            output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
        
        # join bpe split tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(label_idx)
                new_tokens.append(token)
            
        FinalLabelSentence = []
        for token, label in zip(new_tokens, new_labels):
            label = dic_label.get(label)
            if label == "O" or label == "X":
                FinalLabelSentence.append("O")
            else:
                FinalLabelSentence.append(label)
                
        predictedModel.append(FinalLabelSentence[1:-1]) # delete [SEP] and [CLS]
        
            
    return predictedModel

In [6]:
dataset = pd.read_csv('sample_laudos.csv')

In [399]:
test_sentence

'o estudo tomográfico computadorizado do tórax realizado com cortes axiais com a injeção ev de contraste mostra: extenso pneumotórax à direita com colapso do lobo pulmonar ipsilateral e áreas de enfisema subcutâneo arcos costais adjacentes de difícil avaliação por artefato de movimento opacidades pulmonares posteriores (de decúbito) restante do parênquima pulmonar com coeficientes de atenuação conservados. vias aéreas de calibre e aspecto preservados. mediastino centrado sem evidências de massas ou adenomegalias. coração de contornos e dimensões normais. aorta torácica de calibre mantido. estruturas hilares normoconfiguradas. tronco da artéria pulmonar e ramos principais direito e esquerdo pérvios de calibre mantido.'

In [397]:
# THE MODEL ACCEPTS ONLY LOWER
test_sentence = sent_to_words(dataset['laudo_completo'][70])
test_sentence1 = test_sentence.split()

In [394]:
predictedModels = []
for model_name in MODELS_DIR:
    try:   

        #tag_values = tags_values[i]

        #print(tag_values)
        ner_results = predictBERTNER([test_sentence], model_name)
        predictedModels.append(ner_results[0])

    except:
        print('Erro BERT (frase muito grande provavelmente)')
        raise

In [395]:
print(predictedModels)

[['O', 'B-DiagnosticProcedure', 'I-DiagnosticProcedure', 'I-DiagnosticProcedure', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-DiseaseO

In [396]:
len(predictedModels[0])

110